# Trying to understand, from the weights, how a single attn_out feature comes about

I mean this feature is literally after a single attention layer. It's just a direction. How hard could it be?? (I couldn't do it :())

In [1]:
#Load in all the code
#You might need to run like git submodule --init or something to get the submodules
import os
import sys
current_dir = os.path.abspath('.')
print(current_dir)
sys.path.append(current_dir + '/bilinear_interp_tim')    
sys.path.append(current_dir + '/dictionary_learning')
sys.path.append(current_dir)
import argparse
import gc
import json
import math
from collections import defaultdict
import torch as t
from einops import rearrange, einsum
from tqdm import tqdm

from activation_utils import SparseAct
from attribution import patching_effect, jvp
from circuit_plotting import plot_circuit, plot_circuit_posaligned
from dictionary_learning import AutoEncoder
from loading_utils import load_examples, load_examples_nopair
from nnsight import LanguageModel
from language import Transformer, Sight
from sae_adopter import DictionarySAE
from bilinear_circuits_v0 import initialize_model_and_dictionaries
import numpy as np
import pandas as pd

c:\Users\Tim\Documents\bilinear-feature-circuits


In [2]:
from typing import Callable
import einops

In [3]:
device, model, embed, attns, mlps, resids, dictionaries, save_basename, examples, batch_size, num_examples, n_batches, batches = initialize_model_and_dictionaries(
    device='cuda:0',
    model_name="tdooms/fw-nano",
    dict_id='10',  # Note: This was originally an int, but the function expects a string.
    d_model=1024,
    dict_path='tdooms/fw-nano-scope',
    dataset='simple_train',
    num_examples=20,
    example_length=None,
    batch_size=4,
    aggregation='sum',
    nopair=False,
)
# We don't actually need a lot of this stuff

Clean and patch inputs of different shapes.
Clean: 3 Patch: 4
Clean and patch inputs of different shapes.
Clean: 4 Patch: 3


In [4]:
#Picked some subset of fineweb-edu where the model achieves very good loss on some of the tokens
input_tensor = t.tensor(model.tokenizer("A trigger is designed to activate a task of the virus, as displaying strange messages, deleting files, sending emails, begin the replicate process or whatever the programmer write in his malicious code.")['input_ids'], device=device).unsqueeze(0)
input_tensor


tensor([[    1,   330,  8366,   349,  5682,   298, 27854,   264,  3638,   302,
           272, 15022, 28725,   390,  4249,   288,  8708,  8570, 28725, 21750,
           288,  5373, 28725, 10313, 19863, 28725,  2839,   272,   312, 13112,
          1759,   442,  5681,   272,  2007,   794,  3324,   297,   516,  6125,
         10573,  2696, 28723]], device='cuda:0')

In [5]:
with t.no_grad():
    basic_out = model._model.forward(input_ids = input_tensor, labels = input_tensor)
print(basic_out.loss)

tensor(4.6315, device='cuda:0')


In [6]:
#get the per token loss
per_tok_loss = t.log_softmax(basic_out.logits[0,:-1,:], dim = -1).gather(dim = -1, index = input_tensor[0,1:].unsqueeze(-1)).squeeze(-1)
#quick dataframe with what every token is and the next token that the loss is on
df = pd.DataFrame({'current_token': [model.tokenizer.decode(s) for s in input_tensor[0, :-1].cpu().numpy()], 
                   'current_token_id': input_tensor[0, :-1].cpu().numpy(),
                   'next_token': [model.tokenizer.decode(s) for s in input_tensor[0, 1:].cpu().numpy()],
                   'next_token_id': input_tensor[0, 1:].cpu().numpy(),
                     'loss': per_tok_loss.cpu().numpy()})
df


,current_token,current_token_id,next_token,next_token_id,loss
0,<s>,1,A,330,-3.425913
1,A,330,trigger,8366,-10.745537
2,trigger,8366,is,349,-1.523020
3,is,349,designed,5682,-7.466187
4,designed,5682,to,298,-0.119675
5,to,298,activate,27854,-3.481380
6,activate,27854,a,264,-1.152386
7,a,264,task,3638,-6.570916
8,task,3638,of,302,-5.884786
9,of,302,the,272,-1.444378


In [7]:
all_submods = [embed] + [submod for layer_submods in zip(mlps, attns, resids) for submod in layer_submods]
#all_submods[2] is the first attention layer and the one we're interested in. 

#We're going to look at the effect on the single correct token logit: the "to" token after "A trigger is designed"
def single_tok_logit_metric(tok_ind:int) -> Callable[[LanguageModel],t.Tensor]:
    def metric_fn(model: LanguageModel):
        return t.gather(model.lm_head.output[:,-1,:], dim=-1, index=t.tensor([tok_ind], device=model.device).view(-1, 1)).squeeze(-1)
    return metric_fn
short_input = input_tensor[:, :5]

In [8]:
effects, deltas, grads, total_effect = patching_effect(
        clean = short_input,
        patch = None,
        model = model,
        submodules = all_submods,
        dictionaries = dictionaries,
        metric_fn = single_tok_logit_metric(298),
        metric_kwargs=dict(),
        method='ig' # get better approximations for early layers by using ig
    )
#Use integrated gradients to get the effect of each SAE latent on the logit of the "to" token


Integrated Gradient estimation



You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



Initial trace


Patching part



In [12]:
t.cuda.empty_cache()
t.set_printoptions(precision=3, sci_mode=False)

In [13]:
for i, m in enumerate(effects):
    print(f"At submodule {i}")
    print(m)
    print(t.topk(effects[m].act[0,-1], k = 31))
    print("Error term effect")
    print(effects[m].resc[0,0,-1]) #All pretty low actually

At submodule 0
Embedding(32000, 1024)
torch.return_types.topk(
values=tensor([0.084, 0.044, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000,
        0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000,
        0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000,
        0.000], device='cuda:0'),
indices=tensor([2894, 6113,   67,   66,   60,   56,   65,   63,   39,   38,   34,   37,
          50,   48,   62,   53,   23,   19,   13,   10,    0,    4,    6,    8,
          26,   24,   15,   16,   28,   29,   51], device='cuda:0'))
Error term effect
tensor(0.000, device='cuda:0')
At submodule 1
MLP(
  (w): Bilinear(
    in_features=1024, out_features=8192, bias=True
    (gate): Identity()
  )
  (p): Linear(in_features=4096, out_features=1024, bias=True)
)
torch.return_types.topk(
values=tensor([0.099, 0.090, 0.081, 0.034, 0.003, 0.000, 0.000, 0.000, 0.000, 0.000,
        0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000,
        0

In [14]:
print(effects[all_submods[2]].act.shape)
print(t.topk(effects[all_submods[2]].act[0], k = 31)) #we're going to focus on feature 5042, which has a large effect and isn't on every token position

torch.Size([1, 5, 8192])
torch.return_types.topk(
values=tensor([[    0.310,     0.171,     0.144,     0.039,     0.032,     0.027,
             0.024,     0.022,     0.020,     0.009,     0.007,     0.003,
             0.003,     0.001,     0.001,     0.001,     0.000,     0.000,
             0.000,     0.000,     0.000,     0.000,     0.000,     0.000,
             0.000,     0.000,     0.000,     0.000,     0.000,     0.000,
             0.000],
        [    0.020,     0.013,     0.010,     0.006,     0.005,     0.002,
             0.002,     0.001,     0.001,     0.001,     0.000,     0.000,
             0.000,     0.000,     0.000,     0.000,     0.000,     0.000,
             0.000,     0.000,     0.000,     0.000,     0.000,     0.000,
             0.000,     0.000,     0.000,     0.000,     0.000,     0.000,
             0.000],
        [    0.067,     0.041,     0.027,     0.025,     0.018,     0.016,
             0.014,     0.011,     0.011,     0.008,     0.006,     0.006,
 

In [15]:
with model.trace(short_input): #collect some caches
    post_embedding = model._envoy.transformer.h[0].input
    post_embedding.save()
    attn_out = all_submods[2].output
    attn_out.save()
    attn_features = dictionaries[all_submods[2]].encode(attn_out)
    attn_features.save()
    #error term
    error = attn_out - dictionaries[all_submods[2]].decode(attn_features)
    error.save()

print(f"Vector norm {attn_out[0,-1].norm()}, error norm {error[0,-1].norm()}")
    

Vector norm 9.78896713256836, error norm 2.0771453380584717


In [19]:
#Look at the feature activations instead of just the attributions. 5042 is the third largest active sae latent. 
t.topk(attn_features[0,-1], k=31)

torch.return_types.topk(
values=tensor([4.720, 3.842, 3.603, 2.715, 2.527, 1.947, 1.082, 1.031, 0.874, 0.794,
        0.614, 0.548, 0.467, 0.418, 0.417, 0.407, 0.385, 0.366, 0.358, 0.350,
        0.350, 0.339, 0.318, 0.318, 0.315, 0.313, 0.313, 0.302, 0.301, 0.295,
        0.000], device='cuda:0', grad_fn=<TopkBackward0>),
indices=tensor([3514, 6153, 5042,   57,  993, 2764, 3472, 6342, 6874, 6467, 2177, 2452,
        5177, 2891, 5705, 3069, 6496,  693, 1688, 3583, 2440,   64, 7546, 6616,
        6597, 2188, 7805,  617, 8054,  507,   10], device='cuda:0'))

In [20]:
latent_dir3514 = dictionaries[all_submods[2]].w_dec.weight.data[:,3514]
latent_dir5042 = dictionaries[all_submods[2]].w_dec.weight.data[:,5042]
attn_l1 = all_submods[2]

Look at various dot products and norms to get a sense for this latent

In [21]:
t.norm(attn_out[0,-1] - (latent_dir5042 * 3.6028))

tensor(8.874, device='cuda:0', grad_fn=<LinalgVectorNormBackward0>)

In [22]:
t.nn.functional.cosine_similarity(attn_out[0,-1],latent_dir5042 * 3.6028 , dim = 0)

tensor(0.426, device='cuda:0', grad_fn=<SumBackward1>)

In [23]:
einsum(attn_out,latent_dir5042, 'b s dmod, dmod -> b s') #We see that this direction is much more prominent on the last token position

tensor([[-0.297,  0.953, -0.140,  0.634,  4.171]], device='cuda:0',
       grad_fn=<ViewBackward0>)

Instead of learning how to use pytorch hooks like a good person I went and recalculated the intermediate values

In [24]:
#mypy-ignore
with t.no_grad():
    qkv = attn_l1.qkv(post_embedding)
    q, k, v = rearrange(qkv, 'batch seq (n n_head d_head) -> n batch n_head seq d_head', n=3, n_head=16).unbind(dim=0)
    q, k = attn_l1.rotary(q, k)
    scores = einsum(q, k, "batch n_head seq_q d_head, batch n_head seq_k d_head -> batch n_head seq_q seq_k")
    scores = scores / (t.tensor(q.size(-1), device=post_embedding.device).sqrt())
    scores = scores.masked_fill(attn_l1.mask[:,:,:post_embedding.size(1),:post_embedding.size(1)].to(scores.device) == 0, -t.inf)
    pattern = attn_l1.softmax(scores)
    z = einsum(pattern, v, "batch n_head seq_q seq_k, batch n_head seq_k d_head -> batch n_head seq_q d_head")
    o_tensor = rearrange(attn_l1.o.weight, 'd_model (n_head d_head) -> n_head d_model d_head', n_head=16)
    fin_out = einsum(o_tensor, z,'n_head d_model d_head, batch n_head seq d_head -> batch seq n_head d_model' )

In [26]:
t.max(t.abs(einops.reduce(fin_out,'batch seq n_head d_model ->batch seq d_model', 'sum') - attn_out))*100000 #check that this is calculated correctly

tensor(0.048, device='cuda:0', grad_fn=<MulBackward0>)

In [27]:
#Now we can do direct logit attribution but to that specific feature
#finout is batch seq n_head d_model
einsum(fin_out, latent_dir5042, 'batch seq n_head d_model, d_model -> batch seq n_head')

tensor([[[     0.132,     -0.026,     -0.011,      0.014,     -0.089,
              -0.060,      0.030,      0.003,     -0.026,     -0.036,
              -0.007,      0.109,      0.071,     -0.526,      0.023,
               0.103],
         [     0.109,     -0.012,      0.230,      0.004,      0.377,
              -0.001,     -0.123,      0.011,     -0.026,      0.019,
               0.095,      0.057,      0.071,     -0.093,      0.034,
               0.202],
         [     0.069,      0.002,     -0.022,      0.010,     -0.371,
              -0.045,     -0.230,      0.038,     -0.002,     -0.036,
               0.059,     -0.067,      0.026,      0.069,      0.090,
               0.270],
         [     0.033,     -0.007,      0.546,      0.016,     -0.037,
              -0.065,     -0.017,      0.019,     -0.020,     -0.008,
               0.030,     -0.251,      0.056,      0.072,      0.013,
               0.255],
         [     0.165,      0.024,      0.043,      0.029,      1.171

We see that the two main head that are responsible are head 4 and head 14 (or -2). Let's take a look at their attention patterns

In [28]:
pattern[0,4]

tensor([[1.000, 0.000, 0.000, 0.000, 0.000],
        [0.213, 0.787, 0.000, 0.000, 0.000],
        [0.348, 0.032, 0.619, 0.000, 0.000],
        [0.849, 0.014, 0.041, 0.096, 0.000],
        [0.243, 0.010, 0.029, 0.004, 0.713]], device='cuda:0')

In [29]:
pattern[0,14]

tensor([[1.000, 0.000, 0.000, 0.000, 0.000],
        [0.445, 0.555, 0.000, 0.000, 0.000],
        [0.300, 0.036, 0.664, 0.000, 0.000],
        [0.618, 0.217, 0.054, 0.110, 0.000],
        [0.210, 0.017, 0.024, 0.014, 0.735]], device='cuda:0')

In [30]:
#Can also look at all the patterns
pattern

tensor([[[[    1.000,     0.000,     0.000,     0.000,     0.000],
          [    0.874,     0.126,     0.000,     0.000,     0.000],
          [    0.500,     0.349,     0.151,     0.000,     0.000],
          [    0.206,     0.649,     0.075,     0.070,     0.000],
          [    0.451,     0.101,     0.089,     0.137,     0.222]],

         [[    1.000,     0.000,     0.000,     0.000,     0.000],
          [    0.955,     0.045,     0.000,     0.000,     0.000],
          [    0.853,     0.073,     0.075,     0.000,     0.000],
          [    0.918,     0.033,     0.023,     0.026,     0.000],
          [    0.749,     0.022,     0.053,     0.103,     0.074]],

         [[    1.000,     0.000,     0.000,     0.000,     0.000],
          [    0.023,     0.977,     0.000,     0.000,     0.000],
          [    0.614,     0.129,     0.257,     0.000,     0.000],
          [    0.002,     0.000,     0.000,     0.998,     0.000],
          [    0.333,     0.066,     0.215,     0.061,    

Leaving the attention pattern aside, I wanted to decompose the ov circuit and see if low rank approximations of the OV matrix can help me understand how this feature is calculated, and what other inputs could trigger the same feature (ignoring the bias for now, which should be ok)

The idea is that you do A = USV^T, and then you find some U (which is in the column space) that's similar to latent_dir5042, and then you look for the cooresponding V.

My initial guess is that the V would have high dot product with a bunch of verbs in past tense that often comes before "to".

We'll start with head #14, which had the highest DLA score

In [31]:
U, S, Vh = t.linalg.svd(model.ov[0,-2])

In [32]:
S

tensor([    2.505,     1.884,     1.761,  ...,     0.000,     0.000,
            0.000], device='cuda:0', grad_fn=<LinalgSvdBackward0>)

Here I do a bunch of stuff to convince myself I did the SVD correctly

In [33]:
U @ t.diag_embed(S) @ Vh @ Vh[2,:]

tensor([ 0.019,  0.059,  0.011,  ...,  0.050,  0.029, -0.051], device='cuda:0',
       grad_fn=<MvBackward0>)

In [34]:
model.ov[0,-2] @ Vh[2,:]

tensor([ 0.019,  0.059,  0.011,  ...,  0.050,  0.029, -0.051], device='cuda:0',
       grad_fn=<MvBackward0>)

Now I look for plausible low rank approximations to the OV matrix that's relevant for my specific feature.

Reminder that only the absolute cosine similarity matters, since U and V are unique up to a sign change.

In [38]:
for i in range(10):
    print(f"Singular vector {i} with value {S[i].detach().cpu().item():2f}")
    print(f"Cosine similarity with the latent:{t.nn.functional.cosine_similarity(latent_dir5042, U[:,i], dim = 0)}")

Singular vector 0 with value 2.505485
Cosine similarity with the latent:-0.03854630887508392
Singular vector 1 with value 1.884241
Cosine similarity with the latent:-0.0847596526145935
Singular vector 2 with value 1.761036
Cosine similarity with the latent:-0.23725509643554688
Singular vector 3 with value 1.668547
Cosine similarity with the latent:-0.03744002431631088
Singular vector 4 with value 1.622306
Cosine similarity with the latent:-0.04328746348619461
Singular vector 5 with value 1.570366
Cosine similarity with the latent:-0.03828049078583717
Singular vector 6 with value 1.558665
Cosine similarity with the latent:0.029338840395212173
Singular vector 7 with value 1.411765
Cosine similarity with the latent:0.04809585586190224
Singular vector 8 with value 1.362805
Cosine similarity with the latent:-0.06523540616035461
Singular vector 9 with value 1.231059
Cosine similarity with the latent:0.02344028651714325


In [39]:
t.nn.functional.cosine_similarity(latent_dir5042, model.ov[0,-2] @ Vh[2,:], dim = -1).abs()

tensor(0.237, device='cuda:0', grad_fn=<AbsBackward0>)

Let's try singular vector 2. We can see how much the inputs dot product with it, and calculate the corresponding contribution to the latent direction from this one low-rank transformation

In [40]:
einops.einsum(post_embedding, Vh[2,:], 'b s dmod, dmod -> b s')

tensor([[ 0.026, -0.213, -0.413, -0.097, -1.213]], device='cuda:0',
       grad_fn=<ViewBackward0>)

In [99]:
t.dot(latent_dir5042,model.ov[0,-2] @ (Vh[2,:] * -1.213))

tensor(0.507, device='cuda:0', grad_fn=<DotBackward0>)

In [101]:
t.dot(latent_dir5042,model.ov[0,-2] @ post_embedding[0,-1])

tensor(1.625, device='cuda:0', grad_fn=<DotBackward0>)

As you can see, it's honest work but it aint much. This singular value component didn't even make up more than half the contribution from that one head! Kinda sad.

Let's look at Vh and see if it correspond to a recogonizable bunch of tokens in the embedding space:

In [42]:
t.topk(einsum(model.w_e.T, Vh[2,:], 'd_vocab d_model, d_model -> d_vocab'), k = 40)

torch.return_types.topk(
values=tensor([2.911, 2.378, 1.551, 1.425, 1.414, 1.380, 1.374, 1.357, 1.301, 1.300,
        1.293, 1.279, 1.269, 1.265, 1.257, 1.247, 1.246, 1.241, 1.220, 1.219,
        1.216, 1.212, 1.191, 1.188, 1.187, 1.181, 1.181, 1.180, 1.174, 1.171,
        1.160, 1.148, 1.148, 1.146, 1.146, 1.142, 1.140, 1.137, 1.131, 1.129],
       device='cuda:0', grad_fn=<TopkBackward0>),
indices=tensor([28725, 28723, 28745,   548, 30943,  2892, 25648, 27146,  4420,   557,
         1777, 26421, 17795,  3159,   562,   369, 21396, 31110, 31044, 18971,
         2750, 19556,  1561, 26366, 10988,  6270, 30671,  8951,  4627, 30612,
        19494, 13121, 31916, 19231, 29397, 28533, 12934, 13064, 17000, 27078],
       device='cuda:0'))

In [45]:
tk_vals = t.topk(einsum(model.w_e.T, Vh[2,:], 'd_vocab d_model, d_model -> d_vocab'), k = 40)
for i in range(tk_vals.values.size(0)):
    print(f"Token {model.tokenizer.decode(tk_vals.indices[i].item())} with value {tk_vals.values[i].item():2f}")

Token , with value 2.910895
Token . with value 2.378175
Token ; with value 1.551426
Token ", with value 1.425128
Token 研 with value 1.414175
Token án with value 1.379538
Token qué with value 1.373690
Token телем with value 1.357427
 with value 1.301352
Token ), with value 1.299810
Token $, with value 1.292855
Token Wahl with value 1.278691
Token чо with value 1.268872
Token ogn with value 1.265384
Token but with value 1.257416
Token that with value 1.247159
Token junto with value 1.245826
Token 격 with value 1.241187
Token 별 with value 1.220372
Token тий with value 1.218560
Token ido with value 1.215914
Token ęd with value 1.211870
Token *, with value 1.191067
Token midt with value 1.187572
Token auer with value 1.187172
Token hs with value 1.180913
Token च with value 1.180857
Token scape with value 1.179776
Token вы with value 1.174117
Token 钟 with value 1.170934
Token curr with value 1.160333
Token imiento with value 1.148400
Token ∪ with value 1.148233
Token ivi with value 1.146211
T

Nope still totally incoherent. I try again with a different head and it's still not great

In [111]:
U4, S4, Vh4 = t.linalg.svd(model.ov[0,4])

In [115]:
for i in range(10):
    print(f"Singular vector {i} with value {S4[i].detach().cpu().item():4f}")
    print(t.nn.functional.cosine_similarity(latent_dir5042, U4[:,i], dim = 0))

Singular vector 0 with value 2.630258
tensor(0.004, device='cuda:0', grad_fn=<SumBackward1>)
Singular vector 1 with value 2.290593
tensor(-0.023, device='cuda:0', grad_fn=<SumBackward1>)
Singular vector 2 with value 2.011209
tensor(-0.053, device='cuda:0', grad_fn=<SumBackward1>)
Singular vector 3 with value 1.875044
tensor(0.118, device='cuda:0', grad_fn=<SumBackward1>)
Singular vector 4 with value 1.730801
tensor(-0.165, device='cuda:0', grad_fn=<SumBackward1>)
Singular vector 5 with value 1.661326
tensor(0.042, device='cuda:0', grad_fn=<SumBackward1>)
Singular vector 6 with value 1.580658
tensor(0.259, device='cuda:0', grad_fn=<SumBackward1>)
Singular vector 7 with value 1.500666
tensor(0.089, device='cuda:0', grad_fn=<SumBackward1>)
Singular vector 8 with value 1.470811
tensor(0.108, device='cuda:0', grad_fn=<SumBackward1>)
Singular vector 9 with value 1.449576
tensor(-0.068, device='cuda:0', grad_fn=<SumBackward1>)


In [119]:
t.nn.functional.cosine_similarity(latent_dir5042, U4[:,6], dim = -1).abs()

tensor(0.259, device='cuda:0', grad_fn=<AbsBackward0>)

In [122]:
t.nn.functional.cosine_similarity(latent_dir5042, model.ov[0,4] @ Vh4[6,:], dim = -1).abs() #yep it's the same thing

tensor(0.259, device='cuda:0', grad_fn=<AbsBackward0>)

In [121]:
einops.einsum(post_embedding, Vh4[6,:], 'b s dmod, dmod -> b s')

tensor([[-0.033,  0.423, -0.356,  1.013,  0.809]], device='cuda:0',
       grad_fn=<ViewBackward0>)

In [126]:
t.dot(latent_dir5042,model.ov[0,4] @ (Vh4[6,:] * 0.809))

tensor(0.331, device='cuda:0', grad_fn=<DotBackward0>)

In [127]:
t.dot(latent_dir5042,U4[:,6] * 0.809 * S4[6])

tensor(0.331, device='cuda:0', grad_fn=<DotBackward0>)

In [ ]:
t.cuda.empty_cache()